In [0]:
%sql
--Total Charge Amount per provider by department
CREATE OR REPLACE TABLE gold.amt_provider AS
 select
   concat(p.firstname, ' ', p.LastName) Provider_Name,
   dd.Name Dept_Name,
   sum(ft.Amount)  total_amount
 from
   gold.fact_transactions ft
   left join gold.dim_provider p on p.ProviderID = ft.FK_ProviderID
   left join gold.dim_department dd on dd.Dept_Id = p.DeptID
   WHERE  concat(p.firstname, ' ', p.LastName) <> '' AND  dd.Name <>''
 group by
   all ;

In [0]:
%sql
--Total Charge Amount per provider by department for each month for year 2024
 CREATE OR REPLACE TABLE gold.amt_provider_mnthly_2024 AS
 select
   concat(p.firstname, ' ', p.LastName) Provider_Name,
   dd.Name Dept_Name,
   date_format(servicedate, 'yyyyMM')  year_month,
   sum(ft.Amount) Total_Charge_Amt,
   sum(ft.paidamount) Total_Paid_Amt
 from
   gold.fact_transactions ft
   left join gold.dim_provider p on p.ProviderID = ft.FK_ProviderID
   left join gold.dim_department dd on dd.Dept_Id = p.DeptID
 where
   year(ft.ServiceDate) = 2024 AND  concat(p.firstname, ' ', p.LastName) <> '' AND  dd.Name <>''
 group by
   all
 order by
   1,
   3;


In [0]:
delta_table = spark.read.format("delta").load("dbfs:/user/hive/warehouse/gold.db/amt_provider")
adls_path = "/mnt/gold/amt_provider"
delta_table.write.format("delta").mode("overwrite").save(adls_path)

In [ ]:
delta_table = spark.read.format("delta").load("dbfs:/user/hive/warehouse/gold.db/amt_provider_mnthly_2024")
adls_path = "/mnt/gold/amt_provider_mnthly_2024"
delta_table.write.format("delta").mode("overwrite").save(adls_path)